In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import pickle

In [ ]:
class MetapathEncoder(nn.Module):
    def __init__(self,
                 in_dim,
                 out_dim, 
                ):
        self.fc = nn.Linear(in_dim, out_dim)

    def forward(self, metapath):
        x = torch.mean(metapath, dim=1)
        return self.fc(x)

class MAGNN(nn.Module):
    def __init__(self,
                 user_features,
                 movie_features,
                 user_metapaths,
                 movie_metapaths,
                 hidden_dim,
                 out_dim,
                 ):
        self.hidden_dim = hidden_dim

        # later take only train/test features and metapaths

        self.user_features = user_features 
        self.movie_features = movie_features

        self.user_metapaths = user_metapaths
        self.movie_metapaths = movie_metapaths

        self.num_user_features = len(user_features[0])
        self.num_movie_features = len(movie_features[0])

        self.user_feature_encoder = nn.Linear(self.num_user_features, hidden_dim)
        self.movie_feature_encoder = nn.Linear(self.num_movie_features, hidden_dim)

        self.metapath_encoder = MetapathEncoder(hidden_dim, hidden_dim)

        self.user_node_embedding = nn.Linear(hidden_dim, out_dim)
        self.movie_node_embedding = nn.Linear(hidden_dim, out_dim)

        self.recommender = nn.Linear(2 * out_dim, 1)

    def forward(self, user_node, movie_node):
        user_metapath_instances = self.user_metapaths[user_node]
        movie_metapath_instances = self.movie_metapaths[movie_node]

        user_aggregated_metapath = torch.zeros_like(self.hidden_dim)
        for metapath in user_metapath_instances:
            metapath_isntance = torch.tensor([
                self.user_feature_encoder(self.user_features[metapath[0]]),
                self.movie_feature_encoder(self.movie_features[metapath[1]]),
                self.user_feature_encoder(self.user_features[metapath[2]])
            ])
            user_aggregated_metapath += self.metapath_encoder(torch.tensor(metapath_isntance))

        movie_aggregated_metapath = torch.zeros_like(self.hidden_dim)
        for metapath in movie_metapath_instances:
            metapath_isntance = torch.tensor([
                self.movie_feature_encoder(self.movie_features[metapath[0]]),
                self.user_feature_encoder(self.user_features[metapath[1]]),
                self.movie_feature_encoder(self.movie_features[metapath[2]]),
            ])
            movie_aggregated_metapath += self.metapath_encoder(metapath_isntance)

        user_embed = F.sigmoid(self.user_node_embedding(user_aggregated_metapath))
        movie_embed = F.sigmoid(self.movie_node_embedding(movie_aggregated_metapath))

        out = self.recommender(torch.cat([user_embed, movie_embed], dim=1))

        return F.sigmoid(out)

In [8]:
path = '../data/preprocessed/'

in_file = open(path + 'user_features.pickle', 'rb')
user_features = pickle.load(in_file)
in_file.close()

in_file = open(path + 'movie_features.pickle', 'rb')
movie_features = pickle.load(in_file)
in_file.close()

in_file = open(path + 'u_m_u_metapath_map.pickle', 'rb')
user_metapaths = pickle.load(in_file)
in_file.close()

in_file = open(path + 'm_u_m_metapath_map.pickle', 'rb')
movie_metapaths = pickle.load(in_file)
in_file.close()

train_pos = np.load(path + 'train_pos.npy')
test_pos = np.load(path + 'test_pos.npy')

In [ ]:
# TODO:
# train negatives and test negatives 

# train loop